In [1]:
# normalize distribution
# interface
# pass playlist get return for every son in playlist

# IMPORT and standard tweaking

In [2]:
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
%matplotlib inline

#visualization option

In [3]:
my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')
sp=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=my_client_id, client_secret=my_client_secret))


In [4]:
# display all columns
pd.set_option('display.max_columns', None)

In [5]:
viral=pd.read_csv(r'data/analysis/viral_charts_songs_2017-2020.csv')

In [6]:
viral.columns

Index(['Position', 'Track Name', 'Artist', 'URL', 'filename', 'ID', 'date',
       'country', 'group', 'duration_ms', 'popularity', 'acousticness',
       'danceability', 'energy', 'key', 'instrumentalness', 'liveness',
       'loudness', 'mode', 'speechiness', 'valence', 'tempo', 'loud_start'],
      dtype='object')

In [7]:
len(viral.ID.unique())

6007

remarks on the features
* 'valence' indicates the 'happiness' the lower the score the sader the music. [refercence](https://community.spotify.com/t5/Content-Questions/Valence-as-a-measure-of-happiness/td-p/4385221)
* mode 0/1 indicates major/minor (dur/moll) 1=Major / 0=Minor

In [8]:
#rename column length to duration to match the kaggle dataframe
viral=viral.rename(columns={'length':'duration_ms'})


#### dtypes tweaking

In [9]:
viral.dtypes

Position              int64
Track Name           object
Artist               object
URL                  object
filename             object
ID                   object
date                 object
country              object
group                object
duration_ms           int64
popularity            int64
acousticness        float64
danceability        float64
energy              float64
key                   int64
instrumentalness    float64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
valence             float64
tempo               float64
loud_start          float64
dtype: object

In [10]:
#date to datetime
viral.date=pd.to_datetime(viral.date, format="%Y-%m-%d", yearfirst=True)

# mode and key are categroical values

#make mode categorical and name 'Major', 'Minor'
viral['mode'] = np.where(viral['mode'].apply(str)=='1','Major','Minor')

#make key categorical and label it more readable
viral['key'] = 'key_'+viral['key'].apply(str)

#introduce numeric column if viral or not 0/1
viral['viral']=1

In [11]:
# mapping country codes to real country names
keys=list(sorted(set(viral.country)))
values=['Argentina','Australia','Brazil','Canada','Germany','Egypt','Spain','United Kingdom','Indonesia','India','Iceland','Italy','Japan','Marocco','Mexico','New Zealand','Russia','Turkey','Ukraine','United States of America','South Africa']
country_dict = dict(zip(keys, values))
viral['country']=viral['country'].map(country_dict) 

In [12]:
len(viral)

13699

# CAN WE PREDICT IF SONGS ARE VIRAL SONGS
* Supervised learning model with K NEAREST NEIGHBORS
* Will the musical features of snongs labeled as viral be distinguishable enough?
* Using a large kaggle song data base as baseline to compare against songs from viral charts
* https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks

#### only want to keep unique songs: 'top50_unique'
* drop date, country, Position, filename, popularity so we can drop songs that are in the data set multiple times
* they arent important for the analysis at hand

In [13]:
len(viral.ID.unique())

6007

In [14]:
viral_unique=viral.drop(['Position','filename','date','country'],axis=1).drop_duplicates(ignore_index=True)

In [15]:
len(viral_unique)

6007

#### Set Up Spotify dataset from Kaggle as baseline to compare against

In [16]:
kaggle=pd.read_csv(r'data/kaggle/data.csv')

In [17]:
#adding column indicating they are not the viral songs
kaggle['group']='base'

kaggle['viral']=0

In [18]:
# mode and key are categroical values

#make mode categorical and name 'Major', 'Minor'
kaggle['mode'] = np.where(kaggle['mode'].apply(str)=='1','Major','Minor')

#make key categorical and label it more readable
kaggle['key'] = 'key_'+kaggle['key'].apply(str)


#### subsetting for years 2016-2020

In [19]:
# only want to keep songs from 2016 to 2020 to avoid accidentally measuring change in music production
kaggle_year=kaggle[kaggle['year'].isin([2019,2020,2021])].reset_index(drop=True)
print('# of records: ',len(base))
#sadly reissues of old songs cannot be filterd out (ex. Frank Sinatra and Joni Mitchel)

NameError: name 'base' is not defined

#### are there any viral charts songs in the kaggle data set?

In [ ]:
base=kaggle_year[-kaggle_year['id'].isin(viral.ID.unique())]
len(base)

# SET UP FOR MODELLING
* create joint dataframe for analysis
* check multicollinearity
* check outliers
* create dummies for categorical variable (ex key & mode)
* standardize continuous variables

### create joint dataframe 
create joint df of viral charts songs and songs from kaggle DB I want to compare against

In [ ]:
# list of relevant columns
continuous_feat=[
            "acousticness",
            "danceability",
            "duration_ms",
            "energy",
            "instrumentalness",
            "liveness",
            "loudness",
            "speechiness",
            "tempo",
            "valence"]

categorical_feat=["key","mode"]

target_feat=['viral']

#### DATA FRAME ORIGINAL VALUES

#### joint dataframe with true values 
named: analysis_true

In [ ]:
#concat viral DF with DF from kaggle that i use as base
analysis_true=pd.concat([viral[target_feat+categorical_feat+continuous_feat],
                        base[target_feat+categorical_feat+continuous_feat]
                        ],
                       axis=0,
                        ignore_index=True)

#### DATA FRAME: SCALED AND STANDARDIZED
#### joint dataframe with standardized continuous variables
named: analysis_stan

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
# fit without transform for later use when applying to new data 
# copy=False parameter overrites the values in the data frame instead of jus returning the standardized data
scaler = StandardScaler(copy=False).fit(analysis_true[continuous_feat])

In [ ]:
analysis_stan=analysis_true

In [ ]:
#standardize the contnuous feature only
analysis_stan[continuous_feat]=pd.DataFrame(StandardScaler(copy=False).fit_transform(analysis_true[continuous_feat]),columns=continuous_feat)

### checking multicollinearity

In [ ]:
#
corr = round(analysis_stan.corr(),3)
f, ax = plt.subplots(figsize=(10, 10))
sns.set_style("darkgrid")
mask = np.triu(np.ones_like(corr, dtype=bool)) # removing the other side of the heatmap
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap
sns.heatmap(corr,mask=mask,cmap=cmap,linewidths=.5,square=True,annot=True)
plt.show()

#### Create dummies of categorical features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
#picking only the categroical vairbles : key and mode
cat = analysis_stan[categorical_feat]
enc = OneHotEncoder()
cat_encoded = pd.DataFrame(enc.fit_transform(cat).toarray(), columns = enc.get_feature_names())

#### Setting up X and y

In [ ]:
X = pd.concat((analysis_stan[continuous_feat], cat_encoded), axis=1)
y=analysis_stan['viral']
X.head()

In [ ]:
y

#### Split test and train


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.4)

# K NEAREST NEIGHBOUR
* distance based algorithms demand that all variables need to be standardized
* they all need to be on the same scale 

### model and predictions

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knear_model = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
knear_pred = knear_model.predict(X_test)

### Evaluate k nearest neighbour w/ score

In [ ]:
print(knear_model.score(X_test, y_test))
print(pd.Series(knear_pred).value_counts())

### Evaluate k nearest neighbours w/ precision, recall and f1

In [ ]:
#evaluate k nearest neighbour: eval_knear
from sklearn.metrics import precision_score, recall_score, f1_score

eval_knear={'Precision': precision_score(y_test, knear_pred, pos_label=1),#knear_pred predicitons of k nearest neighbour
            'Recall': recall_score(y_test, knear_pred, pos_label=1),
            'F1-Score': f1_score(y_test, knear_pred, pos_label=1)}

pd.DataFrame(eval_knear,index=[0]).style.bar(color='lightblue', vmin=0.0, vmax=1.0)

### Evaluate k nearest neighbours w/ Confusion Matrix and heatmap

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, knear_pred)#knear_pred predicitons of k nearest neighbour
print(cf_matrix)


In [ ]:
plt.figure(figsize=(15,10))
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap
sns.set(font_scale=1.2)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap=cmap,xticklabels=[ 'predicted random song', 'predicted viral song'],
            yticklabels=['actually a random song','actually a viral song']).set_title('K NEAREST NEIGHBOUR: CONFUSION MATRIX')
plt.yticks(rotation = 0)
plt.show()


# DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier().fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)


### Evaluate decision tree

In [ ]:
dt_model.score(X_test, y_test)

In [ ]:
#evaluate decision tree : eval_dt
from sklearn.metrics import precision_score, recall_score, f1_score
eval_dt={'Precision': precision_score(y_test, dt_pred, pos_label=1),
            'Recall': recall_score(y_test, dt_pred, pos_label=1),
            'F1-Score': f1_score(y_test, dt_pred, pos_label=1)}

pd.DataFrame(eval_dt,index=[0]).style.bar(color='lightblue', vmin=0.0, vmax=1.0)

In [ ]:
cf_matrix = confusion_matrix(y_test, dt_pred) #dt_pred : predictions decision tree
print('confusion matrix: \n',cf_matrix)
f, ax = plt.subplots(figsize=(15, 10))
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap

akws = {'ha': 'left','va': 'top'}

ax=sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, annot_kws=akws,
            fmt='.2%', cmap=cmap,xticklabels=[ 'predicted random song', 'predicted viral song'],
            yticklabels=['actually a random song','actually a viral song'])
plt.yticks(rotation = 0)
plt.title('DECISION TREE: CONFUSION MATRIX', size=15)
plt.show()

### Feature importance
in decision tree

In [ ]:
# feature importance embedded in y axis
f, ax = plt.subplots(figsize=(25, 8))
ax = sns.barplot(x=X.columns, 
                 y=dt_model.feature_importances_).set_xticklabels(ax.get_xticklabels(), 
                          rotation=45, 
                          horizontalalignment='right', size = 15)
plt.title('Decision Tree: Feature Importance', size=20)
plt.show()


In [ ]:
pd.DataFrame(dict(zip(X.columns, dt_model.feature_importances_)),index=[0]).T.rename(columns={0:'importance'}).sort_values(by='importance',ascending=False).head().style.bar(color='lightblue',vmin=0.0, vmax=1.0)

# RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier().fit(X_train, y_train)
forest_pred = forest_model.predict(X_test)


### Evaluate Random Forest

In [ ]:
forest_model.score(X_test, y_test)

In [ ]:
#evaluate decision tree : eval_dt
from sklearn.metrics import precision_score, recall_score, f1_score
eval_forest={'Precision': precision_score(y_test, forest_pred, pos_label=1),
            'Recall': recall_score(y_test, forest_pred, pos_label=1),
            'F1-Score': f1_score(y_test, forest_pred, pos_label=1)}

pd.DataFrame(eval_forest,index=[0]).style.bar(color='lightblue', vmin=0.0, vmax=1.0)

In [ ]:
cf_matrix = confusion_matrix(y_test, forest_pred) #dforest_pred : predictions forest tree
f, ax = plt.subplots(figsize=(15, 10))
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap

akws = {'ha': 'left','va': 'top'}

ax=sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, annot_kws=akws,
            fmt='.2%', cmap=cmap,xticklabels=[ 'predicted random song', 'predicted viral song'],
            yticklabels=['actually a random song','actually a viral song'])
plt.yticks(rotation = 0)
plt.title('Forest Tree : CONFUSION MATRIX', size=15)
plt.show()

### Feature importance
in decision tree

In [ ]:
# feature importance embedded in y axis
f, ax = plt.subplots(figsize=(25, 8))
ax = sns.barplot(x=X.columns, 
                 y=forest_model.feature_importances_).set_xticklabels(ax.get_xticklabels(), 
                          rotation=45, 
                          horizontalalignment='right', size = 15)
plt.title('Forest Tree: Feature Importance', size=20)
plt.show()


# SVM SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model = SVC(probability=True).fit(X_train, y_train) # to return propability of classification rather than prediction
svm_pred = svm_model.predict(X_test)


### Evaluate Random Forest

In [ ]:
print(svm_model.score(X_test, y_test))
print(pd.Series(svm_pred).value_counts())

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
eval_forest={'Precision': precision_score(y_test, forest_pred, pos_label=1),
            'Recall': recall_score(y_test, svm_pred, pos_label=1),
            'F1-Score': f1_score(y_test, svm_pred, pos_label=1)}

pd.DataFrame(eval_forest,index=[0]).style.bar(color='lightblue', vmin=0.0, vmax=1.0)

In [ ]:
cf_matrix = confusion_matrix(y_test, svm_pred) #dforest_pred : predictions forest tree
f, ax = plt.subplots(figsize=(15, 10))
cmap = sns.diverging_palette(230, 20, as_cmap=True) #preparing cmap

akws = {'ha': 'left','va': 'top'}

ax=sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, annot_kws=akws,
            fmt='.2%', cmap=cmap,xticklabels=[ 'predicted random song', 'predicted viral song'],
            yticklabels=['actually a random song','actually a viral song'])
plt.yticks(rotation = 0)
plt.title('Support Vector Machine : CONFUSION MATRIX', size=15)
plt.show()

### Feature importance
in decision tree

In [ ]:
# feature importance embedded in y axis
f, ax = plt.subplots(figsize=(25, 8))
ax = sns.barplot(x=X.columns, 
                 y=forest_model.feature_importances_).set_xticklabels(ax.get_xticklabels(), 
                          rotation=45, 
                          horizontalalignment='right', size = 15)
plt.title('Support Vector Machine: Feature Importance', size=20)
plt.show()

#  RADAR PLOT
### FEATURE MEANS OF VIRAL SONGS VERSUS DER FEATURE MEANS OF KAGGLE SONGS
reference: https://www.kaggle.com/typewind/draw-a-radar-chart-with-python-in-a-simple-way

In [ ]:
labels=np.array(analysis_true[continuous_feat].columns)
viral_songs_stats=analysis_true[continuous_feat][analysis_true['viral']==1].mean().values

###
base_songs_stats=analysis_true[continuous_feat][analysis_true['viral']==0].mean().values


In [ ]:
angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
angles=np.concatenate((angles,[angles[0]]))
###
viral_songs_stats=np.concatenate((viral_songs_stats,[viral_songs_stats[0]]))
base_songs_stats=np.concatenate((base_songs_stats,[base_songs_stats[0]]))

In [ ]:
fig=plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, polar=True)
plot1=ax.plot(angles, base_songs_stats, '.', linewidth=2, label='base songs',color='blue')
plot2=ax.plot(angles, viral_songs_stats, '.', linewidth=2, label='viral songs',color='red')
ax.fill(angles, base_songs_stats, alpha=0.25, color='blue')
ax.fill(angles, viral_songs_stats, alpha=0.25, color='red')
ax.set_thetagrids((angles * 180/np.pi)[0:10], labels)
plt.legend(title='Legend', loc='upper left')
ax.set_title('AVERAGE\nVIRAL SONGS vs BASELINE SONGS\n\n',size=20)
ax.grid(True)

In [ ]:
#################################################

# RUN PREDICTOR

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import os
import re

In [ ]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

In [22]:
def predict_hit(url,continuous_feat,categorical_feat):
    
    ###
    
    my_client_id=os.getenv('spoti_CLIENT_ID')
    my_client_secret=os.getenv('spoti_CLIENT_SECRET')
    sp=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=my_client_id, client_secret=my_client_secret))

    ###
    
    meta_feat=['ID','Track_Name','Artist','Audio']
    
    #set up to call spotify API
    song_id=re.findall(r'\d\w+',str(url))[0]
    
    meta = sp.track(song_id)
    features = sp.audio_features(song_id)
    analysis= sp.audio_analysis(song_id)
    
    #create dictionary
    keys=[i for i in meta_feat+continuous_feat+categorical_feat] #these lists should contain the nemes of all features I need
    values=[song_id,
            meta['name'],#song title
            meta['artists'][0]['name'],#artist name
            meta['preview_url'], #audio sinppet 30secs
            features[0]['acousticness'],
            features[0]['danceability'],
            meta['duration_ms'],
            features[0]['energy'],
            features[0]['instrumentalness'],
            features[0]['liveness'],
            features[0]['loudness'],
            features[0]['speechiness'],
            features[0]['tempo'],
            features[0]['valence'],
            features[0]['key'],
            features[0]['mode']]

    df=pd.DataFrame(dict(zip(keys,values)),index=[0])
    
    #standardize the contnuous feature only: using the scaler i fitted with all continuouse variables
    df[continuous_feat]=pd.DataFrame(scaler.transform(df[continuous_feat]),columns=continuous_feat)
    
    #recreate dummies for keys
    for i in range(0,12):
        if df['key'][0] == i:
            df[f'x0_key_{i}']=1.0
        else:
            df[f'x0_key_{i}']=0.0
    df.drop('key',axis=1,inplace=True)
    
    #recreate dummies for keys
    df['x1_Major']=np.where(df['mode'] == 1,1.0,0.0)
    df['x1_Minor']=np.where(df['mode'] == 0,1.0,0.0)
    df.drop('mode',axis=1, inplace=True)

    X_NEW=df.drop(['ID','Track_Name','Artist','Audio'],axis=1)
    
    #MAKING PREDICTION
    dt_result=dt_model.predict(X_NEW)
    knear_result=round(knear_model.predict_proba(X_NEW)[0][1],2)
    forest_probability=round(forest_model.predict_proba(X_NEW)[0][1],2) #probablitiy positiv
    svm_probability=round(svm_model.predict_proba(X_NEW)[0][1],2)
    
    #RADAR PLOT OF GIVEN SONGS
    
    labels=np.array(continuous_feat)
    stats=df.loc[0,labels].values
    
    angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
    # close the plot
    stats=np.concatenate((stats,[stats[0]]))
    angles=np.concatenate((angles,[angles[0]]))

    fig=plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, polar=True)
    ax.plot(angles, stats, 'o-', linewidth=1, color='white')
    ax.fill(angles, stats, alpha=0.25,color='red')
    ax.set_thetagrids((angles * 180/np.pi)[0:10], labels)
    ax.set_title(f"{meta['artists'][0]['name']} - {meta['name']}")
    ax.grid(True)
    
    
    #RETURN
    results_dic={'Artist':df['Artist'][0],
                 'Song':df['Track_Name'][0],
                 'DecisionTree': np.where(dt_result==1,'VIRAL','FAILURE'),
                 'K-Near':f'{round(knear_result*100,2)}%',
                 'RandForest':f'{round(forest_probability*100,2)}%',
                 'SVM-Model':f'{round(svm_probability*100,2)}%',
                 'Audio':df['Audio'][0],
                }
       
    print('\n------')
    return pd.DataFrame(results_dic)

In [24]:
print('Paste URL of Spotify Song\nShould look like this: https://open.spotify.com/track/7qzftATxwdp8G78aMmizXN')
url=input().strip(' ')
predict_hit(url,continuous_feat,categorical_feat)

Paste URL of Spotify Song
Should look like this: https://open.spotify.com/track/7qzftATxwdp8G78aMmizXN


KeyboardInterrupt: Interrupted by user

In [ ]:
########################################

# Songs predicted a viral

US viral charts: https://open.spotify.com/playlist/37i9dQZEVXbKuaTI1Z1Afx

ambiguise
* https://open.spotify.com/track/4pBhTGnL5N5KqsyqU58jee&si=48fdc4e4ae8b4d25
* https://open.spotify.com/track/5mHrHnIqsPoB5eek6FcYfm&si=c88657fa9a684d3c
* Lil Pump https://open.spotify.com/track/43ZyHQITOjhciSUUNPVRHc?si=61YOoQNmRZeTQaeTlEqArg
---
winner
* currently in german viral charts https://open.spotify.com/track/5zPT86unoFaNP2J348n1p0&si=3f34e27a90584902
* Vossi Bop Stormzy https://open.spotify.com/track/5DXKtoZLm31msT7tNGNHLG&si=74197ab0140b4678
* Cradle: https://open.spotify.com/track/18A7ha5BitZjmdHTCwXFbU
* Airwaves https://open.spotify.com/track/5CqkgDH8QZjSqqI3HmYxDD&si=e7b286fbe9cd46d9
* Capital Bra https://open.spotify.com/track/0Dlcu2fgHklOnajVu2dUNU&si=d7f89487035e4e63
* Apache Fame https://open.spotify.com/track/0QjjaCaXE45mvhCnV3C0TA&si=31f9ea9d3c2443bf
* The Weekend https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b&si=593d03f738ad4d58
* Perfume Geniuse https://open.spotify.com/track/6imeB9iHkhAj0Q5sAAANB3&si=6cd7dd154b2c46b8

funny losers
* Love will tear us apart https://open.spotify.com/track/34iOH7LY3vme5rQxsVILZ4&si=051da9cc0c324f8b

In [ ]:
###################

# World in Sounds

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world=world.rename(columns={'name':'country'})
world.head()

In [ ]:
#harmonize country names between world df and viral df

In [ ]:
 list(viral.country) in list(world.country)


In [ ]:
world.country.unique()

In [ ]:
viral.country.unique()
#sorted(list(viral.country.unique()))

In [ ]:
# merge on country
viral_map=world.merge(viral, on='country', how='right')
viral_map.head()

In [ ]:
#mapping variables
from mpl_toolkits.axes_grid1 import make_axes_locatable #to adapt size of legend

fig, ax = plt.subplots(1,1,figsize=(20,15))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
viral_map.plot(ax=ax, column='danceability',cmap='Blues',legend=True,cax=cax)
ax.set(xlabel='Longitude(Degrees)',ylabel='Latitude(Degrees)',title='Viral Map: Danceability')
plt.show()

https://www.kaggle.com/asimislam/tutorial-python-subplots

In [ ]:
#  Numerical Data
#continuous_feat

#  plot Numerical Data
a = 5  # number of rows
b = 2  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(14,8))

for i in continuous_feat:
    ax=plt.subplot(a, b, c)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    viral_map.plot(ax=ax, column=i,cmap='Blues',legend=True,cax=cax)
    ax.set(xlabel='Longitude(Degrees)',ylabel='Latitude(Degrees)',title=f'Viral Map: {i}')
plt.show()

# NOTES

PLOTS
https://seaborn.pydata.org/tutorial/distributions.html

In [ ]:
"""    
    if result==1:
        return print(f"YEAH! THE COMMITTEE OF MEME CATS APPROVES!\n„{(df['Track_Name'][0])}“ by {df['Artist'][0].upper()} will go viral\n------\n„{(df['Track_Name'][0])}“ by {df['Artist'][0].upper()} meets {round(probability*100,2)}% of this model's viral qualities")
    else:
        return print(f"NOPE! The song „{(df['Track_Name'][0])}“ doesn't have what it takes - {df['Artist'][0].upper()} ain't on the meme-team\n------\n„{(df['Track_Name'][0])}“ by {df['Artist'][0].upper()} only meets this model's viral qualities by {round(probability*100,2)}%")
    
"""